In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
import doubletdetection
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['font.sans-serif'] = "Arial"
import scvelo as scv
scv.logging.print_version()
scv.settings.verbosity = 3  # show errors(0), warnings(1), info(2), hints(3)
scv.settings.presenter_view = True  # set max width size for presenter view
scv.settings.set_figure_params('scvelo')  # for beautified visualizationscv.settings.set_figure_params('scvelo') 
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))

Running scvelo 0.2.4 (python 3.9.7) on 2023-02-18 09:29.


ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


In [133]:
adata = sc.read('./adultreanalysis_annatation1221.h5ad') 
adata

AnnData object with n_obs × n_vars = 24967 × 2851
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [134]:
adult_germsubsetforscvelo = adata[adata.obs['cell_type'].isin(['Spermatogonia', 'Spermatocytes', 'Round spermatids', 'Elongating spermatids'])].copy()
adult_germsubsetforscvelo

AnnData object with n_obs × n_vars = 3868 × 2851
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [7]:
## from scipy.sparse import csr_matrix
germ_mat=pd.DataFrame(data=adult_germsubsetforscvelo.X.todense(),index=adult_germsubsetforscvelo.obs_names,columns=adult_germsubsetforscvelo.var_names)

# 解决报错“AttributeError: 'numpy.ndarray' object has no attribute 'todense'”，，“from scipy.sparse import csr_matrix；adata_spermatogonia_only.X = csr_matrix(adata_spermatogonia_only.X)”

In [10]:
germ_mat.to_csv('./figures/germ_matformonocle.csv')

In [11]:
meta = pd.DataFrame(data = adult_germsubsetforscvelo.obs) 
meta.to_csv('./figures/metadataformonocle.tsv',sep="\t")

# 提取原始表达矩阵

In [135]:
adult_germsubsetforscvelo = adult_germsubsetforscvelo.raw.to_adata()
adult_germsubsetforscvelo

AnnData object with n_obs × n_vars = 3868 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [19]:
sc.pp.highly_variable_genes(adult_germsubsetforscvelo, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adult_germsubsetforscvelo, save= ".findhvgsubset1227.pdf")
adult_germsubsetforscvelo.raw = adult_germsubsetforscvelo
adult_germsubsetforscvelo = adult_germsubsetforscvelo[:, adult_germsubsetforscvelo.var.highly_variable]
sc.pp.regress_out(adult_germsubsetforscvelo, ['total_counts', 'pct_counts_mt', 'S_score', 'G2M_score'])
sc.pp.scale(adult_germsubsetforscvelo, max_value=10)

In [20]:
adult_germsubsetforscvelo

AnnData object with n_obs × n_vars = 3868 × 4737
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [21]:
sc.tl.pca(adult_germsubsetforscvelo, svd_solver='arpack')
sc.pp.neighbors(adult_germsubsetforscvelo, n_neighbors=10, n_pcs=40)
sc.tl.tsne(adult_germsubsetforscvelo)
sc.tl.umap(adult_germsubsetforscvelo)
sc.tl.leiden(adult_germsubsetforscvelo,resolution=0.5)

In [22]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(adult_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='adult_germsubsetforscveloleiden1227.pdf')
sc.pl.tsne(adult_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='adult_germsubsetforscveloleiden1227.pdf')

In [23]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(adult_germsubsetforscvelo, color='leiden',legend_loc='on data',save='adult_germsubsetforscveloleiden1227ondata.pdf')

In [24]:
sc.tl.rank_genes_groups(adult_germsubsetforscvelo, 'leiden', method='wilcoxon')

In [25]:
pd.DataFrame(adult_germsubsetforscvelo.uns['rank_genes_groups']['names']).head(5)
adult_germsubsetforscvelo.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = adult_germsubsetforscvelo.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("./figures/diff_rank_genes_groups_adult_germsubsetforscveloleiden1223.csv")

In [228]:
cell_cycle_genes = [x.strip() for x in open('/public/home/zhaox/pigscRNA-seq/cell_cycle_genes (remove).txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]

In [77]:
ax = sc.pl.heatmap(adult_germsubsetforscvelo, s_genes, groupby='leiden', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g1scellcycleadult_germsubsetforscvelo1223.pdf')
ax = sc.pl.heatmap(adult_germsubsetforscvelo, g2m_genes, groupby='leiden', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g2mcellcycleadult_germsubsetforscvelo1223.pdf')

In [26]:
results_file = './figures/adult_germsubsetforscvelo_1223unannotation.h5ad'
adult_germsubsetforscvelo.write(results_file)

In [121]:
adata= sc.read('./figures/adult_germsubsetforscvelo_1223unannotation.h5ad')
adata

AnnData object with n_obs × n_vars = 3627 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [75]:
adult_germsubsetformonocle= adult_germsubsetforscvelo[adult_germsubsetforscvelo.obs['leiden'].isin(['0', '1', '2', '3','4', '5', '6', '7','9','10','11','13', '14'])].copy()
adult_germsubsetformonocle

AnnData object with n_obs × n_vars = 3600 × 4737
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [76]:
adult_germsubsetformonocle = adult_germsubsetformonocle.raw.to_adata()
adult_germsubsetformonocle

AnnData object with n_obs × n_vars = 3600 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [136]:
sc.pp.highly_variable_genes(adult_germsubsetforscvelo, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adult_germsubsetforscvelo, save= ".findhvgsubset1229.pdf")
adult_germsubsetforscvelo.raw = adult_germsubsetforscvelo
adult_germsubsetforscvelo = adult_germsubsetforscvelo[:, adult_germsubsetforscvelo.var.highly_variable]
sc.pp.regress_out(adult_germsubsetforscvelo, ['total_counts', 'pct_counts_mt', 'S_score', 'G2M_score'])
sc.pp.scale(adult_germsubsetforscvelo, max_value=10)
sc.tl.pca(adult_germsubsetforscvelo, svd_solver='arpack')
sc.pp.neighbors(adult_germsubsetforscvelo, n_neighbors=10, n_pcs=40)
sc.tl.tsne(adult_germsubsetforscvelo)
sc.tl.umap(adult_germsubsetforscvelo)
sc.tl.leiden(adult_germsubsetforscvelo,resolution=0.5)
sc.tl.rank_genes_groups(adult_germsubsetforscvelo, 'leiden', method='wilcoxon')
pd.DataFrame(adult_germsubsetforscvelo.uns['rank_genes_groups']['names']).head(5)
adult_germsubsetforscvelo.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = adult_germsubsetforscvelo.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("./figures/diff_rank_genes_groups_adult_germsubsetforscveloleiden1229.csv")
results_file = './figures/adult_germsubsetforscvelo_1229unannotation.h5ad'
adult_germsubsetforscvelo.write(results_file)

In [137]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (4, 4))
sc.pl.umap(adult_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='adult_germsubsetforscveloleiden1229.pdf')
sc.pl.tsne(adult_germsubsetforscvelo, color='leiden',legend_loc='right margin',save='adult_germsubsetforscveloleiden1229.pdf')
sc.pl.umap(adult_germsubsetforscvelo, color='leiden',legend_loc='on data',save='adult_germsubsetforscvelo1229ondata.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [182]:
adult_germsubsetforscvelo.obs.cell_type

AAACCCAAGATGTAGT          P
AAACCCAAGCCGCTTG        Mix
AAACCCAAGGCTCTAT          Z
AAACCCAGTCCAGGTC       SSC1
AAACCCAGTTCGGCGT       SSC1
                     ...   
TTTGGTTTCCATACTT        Mix
TTTGTTGAGACATCAA          D
TTTGTTGAGATGTTAG    others1
TTTGTTGAGTATGGAT         ES
TTTGTTGCATATCTCT          D
Name: cell_type, Length: 3868, dtype: category
Categories (15, object): ['SSC1', 'SSC2', 'Diff SPG1', 'Diff SPG2', ..., 'ES', 'Mix', 'others1', 'others2']

In [80]:
adult_germsubsetformonocle2= adult_germsubsetforscvelo[adult_germsubsetforscvelo.obs['leiden'].isin(['1', '2', '3','4', '5', '6', '7','8','9','10','11'])].copy()
adult_germsubsetformonocle2

AnnData object with n_obs × n_vars = 3000 × 4737
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [183]:
adult_germsubsetforscveloraw = adult_germsubsetforscvelo.raw.to_adata()
adult_germsubsetforscveloraw

AnnData object with n_obs × n_vars = 3868 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [184]:
adult_germsubsetforscveloraw.obs.cell_type

AAACCCAAGATGTAGT          P
AAACCCAAGCCGCTTG        Mix
AAACCCAAGGCTCTAT          Z
AAACCCAGTCCAGGTC       SSC1
AAACCCAGTTCGGCGT       SSC1
                     ...   
TTTGGTTTCCATACTT        Mix
TTTGTTGAGACATCAA          D
TTTGTTGAGATGTTAG    others1
TTTGTTGAGTATGGAT         ES
TTTGTTGCATATCTCT          D
Name: cell_type, Length: 3868, dtype: category
Categories (15, object): ['SSC1', 'SSC2', 'Diff SPG1', 'Diff SPG2', ..., 'ES', 'Mix', 'others1', 'others2']

In [152]:
marker_genes_dict = {
    'SSC': ['UCHL1', 'BNC2', 'UTF1'],
    'Diffing SPG': ['KIT', 'MKI67','SOHLH1','DMRT1'],
    'Diffed SPG': ['STRA8'],
    'L': ['SCML1','SMC1B','ZCWPW1','HERC5','MDK','SPO11','TEX19'],
    'Z': ['SYCP1', 'SYCP3','SYCE1', 'TEX12','TEX101','PRSS50'],
    'P': ['PIWIL1','HORMAD1','MGAT4D','CCDC112','NME8','CCNA1','CCNA2','PROK2','MLLT10','RSPH1'],
    'D': ['SPATA16','CCNB2', 'GYG1','ZC2HC1C','AURKA','TMIGD3','POU5F2','GGN'],
    'RS': ['ADAM2', 'ACRV1','SPACA1','SPACA3','FSIP2','SPINK2'],
    'ES': ['PRM1', 'PRM2', 'TNP1', 'TNP2'],
}

In [153]:
sc.tl.dendrogram(adult_germsubsetforscvelo,groupby="leiden")

In [154]:
sc.pl.dotplot(adult_germsubsetforscvelo, marker_genes_dict, 'leiden', standard_scale='var',dendrogram=True,save="adult_germsubsetforscveloannotation1229.pdf")

categories: 0, 1, 2, etc.
var_group_labels: SSC, Diffing SPG, Diffed SPG, etc.


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


In [176]:
cluster2annotation = {
     '0': 'Mix',
     '1': 'Z',
     '2': 'D',
     '3': 'L',
     '4': 'Diff SPG1',
     '5': 'SSC1',
     '6': 'P',
     '7': 'RS2',
     '8': 'others1',
     '9': 'ES',
     '10': 'Diff SPG2',
     '11': 'RS1',
     '12': 'others2',
     '13': 'SSC2',
     '14': 'RS3',
}
adult_germsubsetforscvelo.obs['cell_type'] = adult_germsubsetforscvelo.obs['leiden'].map(cluster2annotation).astype('category')

In [179]:
original_order = ['Mix', 'Z', 'D', 'L','Diff SPG1','SSC1','P','RS2', 'others1','ES','Diff SPG2','RS1', 'others2', 'SSC2', 'RS3']
adult_germsubsetforscvelo.obs['cell_type'] = adult_germsubsetforscvelo.obs['cell_type'].cat.set_categories(['SSC1', 'SSC2','Diff SPG1','Diff SPG2','L', 'Z','P','D','RS1','RS2','RS3','ES','Mix','others1','others2'])

In [180]:
sc.pl.umap(adult_germsubsetforscvelo, color='cell_type', frameon=True, legend_loc= "right margin", legend_fontsize=10, legend_fontweight="normal", legend_fontoutline=2, save="adultsubsetcelltyperight1229.pdf")

In [230]:
ax = sc.pl.heatmap(adult_germsubsetforscvelo, s_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g1scellcycleadult_germsubsetforscvelo1223.pdf')
ax = sc.pl.heatmap(adult_germsubsetforscvelo, g2m_genes, groupby='cell_type', vmin=-2, vmax=2, cmap='RdBu_r', dendrogram=False, show_gene_labels=False,swap_axes=True, figsize=(11,4),save='g2mcellcycleadult_germsubsetforscvelo1223.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


In [152]:
marker_genes_dict = {
    'SSC': ['UCHL1', 'BNC2', 'UTF1'],
    'Diffing SPG': ['KIT', 'MKI67','SOHLH1','DMRT1'],
    'Diffed SPG': ['STRA8'],
    'L': ['SCML1','SMC1B','ZCWPW1','HERC5','MDK','SPO11','TEX19'],
    'Z': ['SYCP1', 'SYCP3','SYCE1', 'TEX12','TEX101','PRSS50'],
    'P': ['PIWIL1','HORMAD1','MGAT4D','CCDC112','NME8','CCNA1','CCNA2','PROK2','MLLT10','RSPH1'],
    'D': ['SPATA16','CCNB2', 'GYG1','ZC2HC1C','AURKA','TMIGD3','POU5F2','GGN'],
    'RS': ['ADAM2', 'ACRV1','SPACA1','SPACA3','FSIP2','SPINK2'],
    'ES': ['PRM1', 'PRM2', 'TNP1', 'TNP2'],
}

In [234]:
ax = sc.pl.heatmap(adult_germsubsetforscvelo, marker_genes_dict, groupby='cell_type', vmin=-5, vmax=5, cmap='RdBu_r',dendrogram=False, swap_axes=True, figsize=(11,8),save='adultcelltypemarker1229.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


In [235]:
adata

AnnData object with n_obs × n_vars = 3868 × 2000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'terminal_states', 'terminal_states_probs', 'clusters_gradients', 'initial_states', 'initial_states_probs', 'velocity_pseudotime', 'latent_time', 'dpt_pseudotime', 'to_terminal_states_kl_divergence'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood

## 用于cellrank

In [181]:
results_file = './figures/adult_germsubsetforscvelo_1229annotation.h5ad'
adult_germsubsetforscvelo.write(results_file)

In [229]:
adult_germsubsetforscvelo.obs['cell_type']

AAACCCAAGATGTAGT          P
AAACCCAAGCCGCTTG        Mix
AAACCCAAGGCTCTAT          Z
AAACCCAGTCCAGGTC       SSC1
AAACCCAGTTCGGCGT       SSC1
                     ...   
TTTGGTTTCCATACTT        Mix
TTTGTTGAGACATCAA          D
TTTGTTGAGATGTTAG    others1
TTTGTTGAGTATGGAT         ES
TTTGTTGCATATCTCT          D
Name: cell_type, Length: 3868, dtype: category
Categories (15, object): ['SSC1', 'SSC2', 'Diff SPG1', 'Diff SPG2', ..., 'ES', 'Mix', 'others1', 'others2']

In [2]:
###补Figure2图
adata = sc.read('./figures/adult_germsubsetforscvelo_1229annotation.h5ad')
adata

AnnData object with n_obs × n_vars = 3868 × 4737
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [3]:
adata.obs.cell_type

AAACCCAAGATGTAGT          P
AAACCCAAGCCGCTTG        Mix
AAACCCAAGGCTCTAT          Z
AAACCCAGTCCAGGTC       SSC1
AAACCCAGTTCGGCGT       SSC1
                     ...   
TTTGGTTTCCATACTT        Mix
TTTGTTGAGACATCAA          D
TTTGTTGAGATGTTAG    others1
TTTGTTGAGTATGGAT         ES
TTTGTTGCATATCTCT          D
Name: cell_type, Length: 3868, dtype: category
Categories (15, object): ['SSC1', 'SSC2', 'Diff SPG1', 'Diff SPG2', ..., 'ES', 'Mix', 'others1', 'others2']

In [5]:
marker_genes = ['UCHL1','BNC2','DMRT1','SOHLH1', 'ZCWPW1', "TEX101", "NME8",'TMIGD3',"ADAM2", "SPAG16","ACRV1","PRM1",'APOA1','RGS1','IGFBP7']
sc.pl.umap(adata, color = marker_genes, ncols=5, save="figure2adultgermcellmks1229.pdf")

In [7]:
sc.tl.rank_genes_groups(adata, groupby='cell_type', method='wilcoxon')
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(5)
adata.uns['rank_genes_groups'].keys()
#dict_keys(['logfoldchanges', 'names', 'params', 'pvals', 'pvals_adj', 'scores'])
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', 'pvals']}).head(5)
res = pd.DataFrame(    
    {group + '_' + key: result[key][group]    
    for group in groups for key in ['names', 'pvals','logfoldchanges','pvals_adj','scores']})
res.to_csv("diff_rank_genes_groups_adultcelltype1229.csv")

In [12]:
sc.pl.rank_genes_groups_heatmap(adata, use_raw=True, vmin=-5, vmax=5,min_logfoldchange=1,groupby='cell_type', cmap='RdBu_r', show=False,dendrogram=False, save = "fig2germcelluseraw1229.pdf")

{'heatmap_ax': <AxesSubplot:>,
 'groupby_ax': <AxesSubplot:ylabel='cell_type'>,
 'gene_groups_ax': <AxesSubplot:>}

In [10]:
sc.pl.rank_genes_groups_matrixplot(
    adata,
    values_to_plot="logfoldchanges",
    groupby='cell_type',
    cmap='bwr',
    vmin=-4,
    vmax=4,
    show=False,
    dendrogram=False,
    min_logfoldchange=2,
    colorbar_title='log fold change',
    save = "fig2germcellmatrixplot1229.pdf")

{'mainplot_ax': <AxesSubplot:>,
 'gene_group_ax': <AxesSubplot:>,
 'color_legend_ax': <AxesSubplot:title={'center':'log fold change'}>}

# scvelo

In [2]:
adata = sc.read('./figures/adult_germsubsetforscvelo_1223unannotation.h5ad')
adata

AnnData object with n_obs × n_vars = 3868 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

ERROR: XMLRPC request failed [code: -32500]
RuntimeError: PyPI no longer supports 'pip search' (or XML-RPC search). Please use https://pypi.org/search (via a browser) instead. See https://warehouse.pypa.io/api-reference/xml-rpc.html#deprecated-methods for more information.


## 细胞注释（1227）

In [104]:
adult_germsubsetformonocle = adult_germsubsetformonocle.raw.to_adata()
adult_germsubsetformonocle

AnnData object with n_obs × n_vars = 3600 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    obsp: 'connectivities', 'distances'

In [185]:
adata_loom = scv.read('/public/home/zhaox/pigscRNA-seq/Adult/Adult.loom', cache=True)
scv.pl.proportions(adata_loom,save="./figures/proportions1224forscvelo.pdf")
adata = scv.utils.merge(adult_germsubsetforscveloraw, adata_loom)
scv.pl.proportions(
    adata,
    groupby='cell_type',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/germcellforscvelospliced1229.pdf",
)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

saving figure to file ./figures/proportions1224forscvelo.pdf
saving figure to file ./figures/germcellforscvelospliced1229.pdf
Filtered out 9225 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 2000 highly variable genes.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


# Estimate RNA velocity

In [6]:
scv.tl.velocity(adata)

computing velocities
    finished (0:00:01) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)


In [7]:
scv.tl.velocity_graph(adata)

computing velocity graph (using 1/24 cores)


  0%|          | 0/3868 [00:00<?, ?cells/s]

    finished (0:00:18) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [8]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color='leiden',save='./figures/germcellforscvelo_embeddingstream1224.pdf')

computing velocity embedding
    finished (0:00:01) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
figure cannot be saved as pdf, using png instead.
saving figure to file ./figures/germcellforscvelo_embeddingstream1224.png


In [11]:
scv.pl.velocity_embedding(adata, arrow_length=3, arrow_size=2, dpi=600,color='leiden',legend_loc='on data',save='./figures/germcellforscvelo_velocity1224.pdf')

saving figure to file ./figures/germcellforscvelo_velocity1224.pdf


# Interprete the velocities ['UCHL1', 'SOHLH1', 'DMRT1','STRA8','SMC1B','ZCWPW1','ACRV1','SPACA1','PRM1', 'PRM2', 'TNP1', 'TNP2', 'SPATA3']

In [14]:
scv.pl.velocity(adata, ['UCHL1', 'SOHLH1', 'DMRT1','MKI67'],color='leiden', ncols=2,save='./figures/germcellforscvelo_phaseportraitsmarkers1224.pdf')

saving figure to file ./figures/germcellforscvelo_phaseportraitsmarkers1224.pdf


In [13]:
scv.pl.velocity(adata, ['SMC1B','ZCWPW1','ACRV1','SPACA1'],color='leiden', ncols=2,save='./figures/germcellforscvelo_phasesmc1b1224.pdf')

saving figure to file ./figures/germcellforscvelo_phasesmc1b1224.pdf


In [15]:
scv.pl.velocity(adata, ['PRM1', 'PRM2', 'TNP1', 'TNP2', 'SPATA3'],color='leiden', ncols=2,save='./figures/germcellforscvelo_phaseprm1tnp1224.pdf')

saving figure to file ./figures/germcellforscvelo_phaseprm1tnp1224.pdf


# Identify important genes (作图有需要再补充吧)

In [16]:
scv.tl.rank_velocity_genes(adata, groupby='leiden', min_corr=.3)
df = scv.DataFrame(adata.uns['rank_velocity_genes']['names'])
df.head()

ranking velocity genes
    finished (0:00:03) --> added 
    'rank_velocity_genes', sorted scores by group ids (adata.uns) 
    'spearmans_score', spearmans correlation scores (adata.var)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,UTRN,HSF5,FTL,SCP2D1,BBX,CCDC181,CEP128,BCAP31,NME7,ENTPD1,RHOBTB3,SCP2D1,PTPN22,KIAA0895,PRKCQ,ITK,PTPRG,BNC2,ACRV1
1,ADAMTS9,LOC110257476,CLGN,WNK3,ART3,CALR3,SLC9B1,BUD23,CCDC60,FSIP1,CLPB,WNK3,STAT4,LOC100524097,UTRN,RUNX3,FN1,MED12L,PTPRC
2,CRIM1,NSUN7,MKI67,TCHH,DAZL,LOC100519521,WDR62,BBX,CFAP58,LRRIQ1,PRPSAP1,CPEB4,STK39,MGAT4D,NCOA7,STK17A,EBF1,ADAMTSL1,C3H2orf74
3,AFF1,MAATS1,LOC110260932,SLC9B1,ZCWPW1,LOC110261055,SPDYA,GALNTL5,ULK4,LEKR1,HEMGN,HFM1,ARHGEF3,HECTD2,WWTR1,DOCK10,LAMA2,GALNTL5,C1H6orf118
4,SASH1,ARID4A,PRPSAP1,MEIOB,HFM1,MROH2A,HHLA2,STK17A,KIAA0825,SPATA17,FTL,MEIOB,GOLM1,LOC110260932,ZFP36L1,CYTIP,LSMEM1,LOC100524794,SH3YL1


In [17]:
kwargs = dict(frameon=False, size=10, linewidth=1.5)
scv.pl.scatter(adata, df['0'][:10], ylabel='0', color='leiden', ncols=5, save='./figures/germcellforscvelocluster0_genecandidates.pdf',**kwargs)

saving figure to file ./figures/germcellforscvelocluster0_genecandidates.pdf


# Velocities in cycling progenitors
# The cell cycle detected by RNA velocity, is biologically affirmed by cell cycle scores (standardized scores of mean expression levels of phase marker genes).

In [18]:
scv.tl.score_genes_cell_cycle(adata)
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95], save= './figures/germcellforscvelo_cellcyclephase.pdf')

calculating cell cycle phase
-->     'S_score' and 'G2M_score', scores of cell cycle phases (adata.obs)
saving figure to file ./figures/germcellforscvelo_cellcyclephase.pdf


In [20]:
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)
s_genes = scv.get_df(adata[:, s_genes], 'spearmans_score', sort_values=True).index
g2m_genes = scv.get_df(adata[:, g2m_genes], 'spearmans_score', sort_values=True).index

kwargs = dict(frameon=False, ylabel='cell cycle genes')
scv.pl.scatter(adata, list(s_genes[:5]) + list(g2m_genes[:5]),color='leiden', **kwargs,save='./figures/germcellforscvelo_cellcycle_phaseportraits.pdf')

saving figure to file ./figures/germcellforscvelo_cellcycle_phaseportraits.pdf


In [22]:
scv.pl.velocity(adata, ['MKI67','TTK','CENPF'], ncols=2, color='leiden',save='./figures/germcellforscvelo_CENPFMKI67_phaseportraits.pdf')

saving figure to file ./figures/germcellforscvelo_CENPFMKI67_phaseportraits.pdf


# Speed and coherence
# Two more useful stats: - The speed or rate of differentiation is given by the length of the velocity vector. - The coherence of the vector field (i.e., how a velocity vector correlates with its neighboring velocities) provides a measure of confidence.
# (重要) These provide insights where cells differentiate at a slower/faster pace, and where the direction is un-/determined.

In [23]:
scv.tl.velocity_confidence(adata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(adata, c=keys, cmap='coolwarm', color='leiden', perc=[5, 95], save='./figures/germcellforscvelovelocity_lengthconfidence.pdf')

--> added 'velocity_length' (adata.obs)
--> added 'velocity_confidence' (adata.obs)
--> added 'velocity_confidence_transition' (adata.obs)
saving figure to file ./figures/germcellforscvelovelocity_lengthconfidence.pdf


In [24]:
df = adata.obs.groupby('leiden')[keys].mean().T
df.style.background_gradient(cmap='coolwarm', axis=1)

leiden,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
velocity_length,7.824484,10.365988,8.786892,4.577942,5.610000,7.806567,4.106429,3.999681,10.645082,10.401092,18.419329,5.397273,17.444445,5.739756,7.909041,8.519841,10.568709,3.821026,7.180313
velocity_confidence,0.768711,0.868035,0.751566,0.834872,0.936073,0.577020,0.929939,0.769593,0.933925,0.948079,0.880582,0.896084,0.937869,0.885766,0.767653,0.854595,0.862628,0.646432,0.848301


# Velocity graph and pseudotime
# We can visualize the velocity graph to portray all velocity-inferred cell-to-cell connections/transitions. It can be confined to high-probability transitions by setting a threshold. The graph, for instance, indicates two phases of Epsilon cell production, coming from early and late Pre-endocrine cells.

In [25]:
scv.pl.velocity_graph(adata, color='leiden', threshold=.1,save='./figures/germcellforscvelo_velocitygraph.pdf')

saving figure to file ./figures/germcellforscvelo_velocitygraph.pdf


## Further, the graph can be used to draw descendents/anscestors coming from a specified cell. Here, a pre-endocrine cell is traced to its potential fate.（用处不大）

In [26]:
x, y = scv.utils.get_cell_transitions(adata, basis='umap')
ax = scv.pl.velocity_graph(adata, c='lightgrey',color='leiden', edge_width=.05, show=False,save='./figures/germcellforscvelo_celltransitions.pdf')
ax = scv.pl.scatter(adata, x=x, y=y, s=120, c='ascending',color='leiden', cmap='gnuplot', ax=ax,save='./figures/germcellforscvelo_celltransitionsscatter.pdf')

saving figure to file ./figures/germcellforscvelo_celltransitions.pdf
saving figure to file ./figures/germcellforscvelo_celltransitionsscatter.pdf


In [29]:
scv.tl.velocity_pseudotime(adata)
scv.pl.scatter(adata, color=['leiden','velocity_pseudotime'], cmap='gnuplot',save='./figures/germcellforscvelo_velocitypseudotime.pdf')

saving figure to file ./figures/germcellforscvelo_velocitypseudotime.pdf


# PAGA velocity graph

In [30]:
adata.uns['neighbors']['distances'] = adata.obsp['distances']
adata.uns['neighbors']['connectivities'] = adata.obsp['connectivities']

In [31]:
scv.tl.paga(adata, groups='leiden')

running PAGA using priors: ['velocity_pseudotime']
    finished (0:00:01) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [32]:
df = scv.get_df(adata, 'paga/transitions_confidence').T

In [33]:
df.style.background_gradient(cmap='Blues').format('{:.2g}')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,0,0,0.48,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.15,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0.3,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0.92,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0.064,0,0,0,0,0,0,0
5,0.72,0.44,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0.2,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0.16,0,0,0,0,0,0,0,0,0,0


In [34]:
scv.pl.paga(adata, basis='umap', size=50, alpha=.1,
            min_edge_width=2, node_size_scale=1.5,save='./figures/germcellforscvelo_pagavelocitygraph.pdf')

saving figure to file ./figures/germcellforscvelo_pagavelocitygraph.pdf


# Dynamical Modeling
# Here, we use the generalized dynamical model to solve the full transcriptional dynamics. That yields several additional insights such as latent time and identification of putative driver genes

In [35]:
scv.tl.recover_dynamics(adata)

recovering dynamics (using 1/24 cores)


  0%|          | 0/853 [00:00<?, ?gene/s]

    finished (0:06:38) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


In [36]:
scv.tl.velocity(adata, mode='dynamical')
scv.tl.velocity_graph(adata)

computing velocities
    finished (0:00:05) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/24 cores)


  0%|          | 0/3868 [00:00<?, ?cells/s]

    finished (0:00:14) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [37]:
scv.pl.velocity_embedding_stream(adata, basis='umap',color='leiden', save= './figures/germcellforscvelo_embeddingstreamdynamic.pdf')

computing velocity embedding
    finished (0:00:01) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
figure cannot be saved as pdf, using png instead.
saving figure to file ./figures/germcellforscvelo_embeddingstreamdynamic.png


# Kinetic rate paramters
# The rates of RNA transcription, splicing and degradation are estimated without the need of any experimental data. They can be useful to better understand the cell identity and phenotypic heterogeneity.

In [39]:
df = adata.var
df = df[(df['fit_likelihood'] > .1) & df['velocity_genes'] == True]

kwargs = dict(xscale='log', fontsize=16)
with scv.GridSpec(ncols=3) as pl:
    pl.hist(df['fit_alpha'], xlabel='transcription rate', **kwargs)
    pl.hist(df['fit_beta'] * df['fit_scaling'], xlabel='splicing rate', xticks=[.1, .4, 1], **kwargs)
    pl.hist(df['fit_gamma'], xlabel='degradation rate', xticks=[.1, .4, 1], **kwargs)
scv.get_df(adata, 'fit*', dropna=True).head()

,fit_alpha,fit_beta,fit_gamma,fit_t_,fit_scaling,fit_std_u,fit_std_s,fit_likelihood,fit_u0,fit_s0,fit_pval_steady,fit_steady_u,fit_steady_s,fit_variance,fit_alignment_scaling,fit_r2
C1H6orf118,0.143782,0.181840,0.317382,6.039877,2.965552,0.233452,0.099648,0.034424,0.0,0.0,0.445436,0.979616,0.313856,3.405797,2.235585,0.458620
PRKN,0.115051,0.039498,0.476845,8.891260,7.300695,0.724865,0.044679,0.042588,0.0,0.0,0.449999,4.350484,0.199466,2.953957,1.560549,0.292099
TAGAP,0.128937,0.184921,0.398351,7.313920,3.421175,0.171624,0.057021,0.060463,0.0,0.0,0.483758,0.715779,0.220375,1.465149,1.651793,0.441423
RSPH3,0.263800,0.449781,0.229660,11.603554,1.090073,0.211434,0.330706,0.278298,0.0,0.0,0.478431,0.673216,0.772836,0.491326,3.197062,0.327576
MYCT1,0.179147,0.694023,0.323062,3.971321,0.923437,0.075373,0.103320,0.094244,0.0,0.0,0.363849,0.205733,0.329805,1.543527,1.975127,0.281958


In [40]:
plt.savefig("./figures/germcellforscvelo_kineticrateparamters.pdf")

# Latent time
# The dynamical model recovers the latent time of the underlying cellular processes. This latent time represents the cell’s internal clock and approximates the real time experienced by cells as they differentiate, based only on its transcriptional dynamics

In [41]:
scv.tl.latent_time(adata)
scv.pl.scatter(adata, color='latent_time', color_map='gnuplot', size=80, save= './figures/germcellforscvelo_latenttimedynamic.pdf')

computing latent time using root_cells as prior
    finished (0:00:03) --> added 
    'latent_time', shared time (adata.obs)
saving figure to file ./figures/germcellforscvelo_latenttimedynamic.pdf


In [42]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index[:300]
scv.pl.heatmap(adata, var_names=top_genes, sortby='latent_time', col_color='leiden', n_convolve=100, save= './figures/germcellforscvelo_latenttimeheatmapdynamic.pdf')

saving figure to file ./figures/germcellforscvelo_latenttimeheatmapdynamic.pdf


In [45]:
top_genes = adata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(adata, basis=top_genes[:15], ncols=5, frameon=False,color='leiden',save= './figures/germcellforscvelo_topgene15dynamic.pdf')

saving figure to file ./figures/germcellforscvelo_topgene15dynamic.pdf


In [46]:
adata

AnnData object with n_obs × n_vars = 3868 × 2000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'clusters_gradients', 'velocity_length', 'velocity_confidence', 'velocity_confidence_transition', 'root_cells', 'end_points', 'velocity_pseudotime', 'latent_time'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'velocity_gamma', 'velocity_qreg_ratio', 'velocity_r2', 'velocity_genes', 'spearmans_score', 'velocity_score', 'fit_alpha', 'fit_beta', 'fit_gamma', '

## cellrank

In [2]:
pip install cellrank

     |████████████████████████████████| 250 kB 704 kB/s eta 0:00:01
     |████████████████████████████████| 1.8 MB 137 kB/s eta 0:00:01
     |████████████████████████████████| 68 kB 370 kB/s  eta 0:00:01
     |████████████████████████████████| 133 kB 12.2 MB/s eta 0:00:01
  Attempting uninstall: MarkupSafe
    Found existing installation: MarkupSafe 1.1.1
    Uninstalling MarkupSafe-1.1.1:
      Successfully uninstalled MarkupSafe-1.1.1
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.10.1 requires ruamel-yaml, which is not installed.
cookiecutter 1.7.2 requires Jinja2<3.0.0, but you have jinja2 3.0.3 which is incompatible.
cookiecutter 1.7.2 requires MarkupSafe<2.0.0, but you have markupsaf

In [65]:
import scvelo as scv
import scanpy as sc
import cellrank as cr
import numpy as np
scv.settings.verbosity = 3
scv.settings.set_figure_params("scvelo")
cr.settings.verbosity = 2
import warnings
warnings.simplefilter("ignore", category=UserWarning)
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=DeprecationWarning)

In [4]:
adata = sc.read('./figures/adult_germsubsetforscvelo_1223unannotation.h5ad')
adata

AnnData object with n_obs × n_vars = 3868 × 19297
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'dendrogram_cell_type', 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'neighbors', 'pca', 'rank_genes_groups', 'tsne', 'umap'
    obsm: 'X_pca', 'X_tsne', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [89]:
adata_loom = scv.read('/public/home/zhaox/pigscRNA-seq/Adult/Adult.loom', cache=True)
scv.pl.proportions(adata_loom,save="./figures/proportions1225forscvelo.pdf")
adata = scv.utils.merge(adata, adata_loom)
scv.pl.proportions(
    adata,
    groupby='leiden',
    layers=None,
    highlight='unspliced',
    add_labels_pie=True,
    add_labels_bar=True,
    fontsize=8,
    figsize=(15, 2),
    dpi=600,
    use_raw=True,
    show=True,
    save="./figures/germcellforcellrankspliced1225.pdf",
)
scv.pp.filter_and_normalize(adata, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(adata, n_pcs=30, n_neighbors=30)

saving figure to file ./figures/proportions1225forscvelo.pdf
saving figure to file ./figures/germcellforcellrankspliced1225.pdf
Filtered out 9627 genes that are detected 20 counts (shared).
Normalized count data: spliced, unspliced.
Extracted 2000 highly variable genes.
computing neighbors
    finished (0:00:00) --> added 
    'distances' and 'connectivities', weighted adjacency matrices (adata.obsp)
computing moments based on connectivities
    finished (0:00:00) --> added 
    'Ms' and 'Mu', moments of un/spliced abundances (adata.layers)


# Run scVelo
# We will use the dynamical model from scVelo to estimate the velocities. Please make sure to have at least version 0.2.3 of scVelo installed to make use parallelisation in scv.tl.recover_dynamics. On my laptop, using 8 cores, the below cell takes about 1:30 min to execute.

In [186]:
scv.tl.recover_dynamics(adata, n_jobs=8)

recovering dynamics (using 8/24 cores)


  0%|          | 0/1246 [00:00<?, ?gene/s]

    finished (0:01:16) --> added 
    'fit_pars', fitted parameters for splicing dynamics (adata.var)


# Once we have the parameters, we can use these to compute the velocities and the velocity graph. The velocity graph is a weighted graph that specifies how likely two cells are to transition into another, given their velocity vectors and relative positions.

In [187]:
scv.tl.velocity(adata, mode="dynamical")
scv.tl.velocity_graph(adata)

computing velocities
    finished (0:00:09) --> added 
    'velocity', velocity vectors for each individual cell (adata.layers)
computing velocity graph (using 1/24 cores)


  0%|          | 0/3868 [00:00<?, ?cells/s]

    finished (0:00:15) --> added 
    'velocity_graph', sparse matrix with cosine correlations (adata.uns)


In [188]:
scv.pl.velocity_embedding_stream(
    adata, basis="umap",color='cell_type', legend_fontsize=12, title="", smooth=0.8, min_mass=4,save='./figures/germcellforcellrank_embeddingstream1225.pdf'
)

computing velocity embedding
    finished (0:00:01) --> added
    'velocity_umap', embedded velocity vectors (adata.obsm)
figure cannot be saved as pdf, using png instead.
saving figure to file ./figures/germcellforcellrank_embeddingstream1225.png


# Run CellRank
# CellRank offers various ways to infuse directionality into single-cell data. Here, the directional information comes from RNA velocity, and we use this information to compute initial & terminal states as well as fate probabilities for the dynamical process of pancreatic development.

# Identify terminal states
# Terminal states can be computed by running the following command:

In [189]:
cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)

Accessing `adata.obsp['T_fwd']`
Computing transition matrix based on logits using `'deterministic'` mode


/tmp/ipykernel_178141/637574554.py:1: DeprecationWarning: `cellrank.tl.terminal_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.terminal_states(adata, cluster_key="cell_type", weight_connectivities=0.2)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


Estimating `softmax_scale` using `'deterministic'` mode


  0%|          | 0/3868 [00:00<?, ?cell/s]

Setting `softmax_scale=7.0976`


  0%|          | 0/3868 [00:00<?, ?cell/s]

    Finish (0:00:38)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_fwd']`
       `.eigendecomposition`
    Finish (0:00:05)
Computing Schur decomposition
Adding `adata.uns['eigendecomposition_fwd']`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:01:34)
Computing `3` macrostates
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:03)
Adding `adata.obs['terminal_states']`
       `adata.obs['terminal_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`


In [190]:
cr.pl.terminal_states(adata,color='cell_type')

In [191]:
cr.pl.terminal_states(adata,color='cell_type')

In [130]:
??cr.tl.initial_states

Signature:
cr.tl.initial_states(
    adata: anndata._core.anndata.AnnData,
    estimator: abc.ABCMeta = <class 'cellrank.tl.estimators.terminal_states._gpcca.GPCCA'>,
    mode: str = 'deterministic',
    backward_mode: str = 'transpose',
    n_states: Optional[int] = None,
    cluster_key: Optional[str] = None,
    key: Optional[str] = None,
    force_recompute: bool = False,
    show_plots: bool = False,
    copy: bool = False,
    return_estimator: bool = False,
    fit_kwargs: Mapping = mappingproxy({}),
    **kwargs,
) -> Union[anndata._core.anndata.AnnData, cellrank.tl.estimators._base_estimator.BaseEstimator, NoneType]
Docstring:
Find initial states of a dynamic process of single cells based on RNA velocity :cite:`manno:18`.

The function models dynamic cellular processes as a Markov chain, where the transition matrix is computed based
on the velocity vector of each individual cell. Based on this Markov chain, we provide two estimators
to compute initial states, both of which are

# Identify initial states
# The same sort of analysis can now be repeated for the initial states, only that we use the function cr.tl.initial_states this time:

In [192]:
cr.tl.initial_states(adata, cluster_key="cell_type")
cr.pl.initial_states(adata, discrete=True)

Accessing `adata.obsp['T_bwd']`


/tmp/ipykernel_178141/1621579496.py:1: DeprecationWarning: `cellrank.tl.initial_states` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.initial_states(adata, cluster_key="cell_type")
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/cellrank/tl/_init_term_states.py:156: DeprecationWarning: `cellrank.tl.transition_matrix` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  kernel = transition_matrix(


Computing transition matrix based on logits using `'deterministic'` mode
Estimating `softmax_scale` using `'deterministic'` mode


  0%|          | 0/3868 [00:00<?, ?cell/s]

Setting `softmax_scale=7.0976`


  0%|          | 0/3868 [00:00<?, ?cell/s]

    Finish (0:00:19)
Using a connectivity kernel with weight `0.2`
Computing transition matrix based on `adata.obsp['connectivities']`
    Finish (0:00:00)
Computing eigendecomposition of the transition matrix
Adding `adata.uns['eigendecomposition_bwd']`
       `.eigendecomposition`
    Finish (0:00:05)
For 1 macrostate, stationary distribution is computed
Adding `.macrostates`
       `.macrostates_memberships`
       `.coarse_T`
       `.coarse_initial_distribution
       `.coarse_stationary_distribution`
       `.schur_vectors`
       `.schur_matrix`
       `.eigendecomposition`
    Finish (0:00:00)
Adding `adata.obs['initial_states']`
       `adata.obs['initial_states_probs']`
       `.terminal_states`
       `.terminal_states_probabilities`
       `.terminal_states_memberships
    Finish`


In [193]:
plt.savefig('./figures/germcellforcellrank_initial_states1225.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


# Compute fate maps
# Once we know the terminal states, we can compute associated fate maps - for each cell, we ask how likely is the cell to develop towards each of the identified terminal states.

In [194]:
cr.tl.lineages(adata)
cr.pl.lineages(adata, same_plot=False,ncols=2)
plt.savefig('./figures/germcellforcellrank_fatemaps1225.pdf')

Computing absorption probabilities
Defaulting to `'gmres'` solver.


/tmp/ipykernel_178141/977518106.py:1: DeprecationWarning: `cellrank.tl.lineages` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineages(adata)


  0%|          | 0/3 [00:00<?, ?/s]

Adding `adata.obsm['to_terminal_states']`
       `.absorption_probabilities`
    Finish (0:00:01)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


In [227]:
cr.pl.circular_projection(
    adata, keys=["cell_type", "kl_divergence"], legend_loc="upper right"
)
plt.savefig('./figures/germcellforcellrank_circular_projections1225.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


In [225]:
adata

AnnData object with n_obs × n_vars = 3868 × 2000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'terminal_states', 'terminal_states_probs', 'clusters_gradients', 'initial_states', 'initial_states_probs', 'velocity_pseudotime', 'latent_time', 'dpt_pseudotime'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood', 'fit_u0', 'fit_s0', 'fit_pval_ste

# We can aggregate the above into a single, global fate map where we associate each terminal state with color and use the intensity of that color to show the fate of each individual cell:

In [195]:
cr.pl.lineages(adata, same_plot=True)
plt.savefig('./figures/germcellforcellrank_fatemapsintoasingle1225.pdf')

# Directed PAGA
# We can further aggregate the individual fate maps into a cluster-level fate map using an adapted version of [Wolf et al., 2019] with directed edges. We first compute scVelo’s latent time with CellRank identified root_key and end_key, which are the probabilities of being an initial or a terminal state, respectively.

In [196]:
scv.tl.recover_latent_time(
    adata, root_key="initial_states_probs", end_key="terminal_states_probs"
)

computing latent time using initial_states_probs, terminal_states_probs as prior
    finished (0:00:03) --> added 
    'latent_time', shared time (adata.obs)


In [197]:
# Next, we can use the inferred pseudotime along with the initial and terminal states probabilities to compute the directed PAGA.
scv.tl.paga(
    adata,
    groups="cell_type",
    root_key="initial_states_probs",
    end_key="terminal_states_probs",
    use_time_prior="velocity_pseudotime",
)

running PAGA using priors: ['velocity_pseudotime', 'initial_states_probs', 'terminal_states_probs']
    finished (0:00:01) --> added
    'paga/connectivities', connectivities adjacency (adata.uns)
    'paga/connectivities_tree', connectivities subtree (adata.uns)
    'paga/transitions_confidence', velocity transitions (adata.uns)


In [198]:
cr.pl.cluster_fates(
    adata,
    mode="paga_pie",
    cluster_key="cell_type",
    basis="umap",
    legend_kwargs={"loc": "top right out"},
    legend_loc="top left out",
    node_size_scale=5,
    dpi=600,
    edge_width_scale=1,
    max_edge_width=4,
    title="directed PAGA",
)
plt.savefig('./figures/germcellforcellrank_clusterfates1225.pdf')

In [199]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (8, 4))
plt.savefig('./figures/germcellforcellrank_clusterfates1225.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)


In [221]:
cr.pl.cluster_fates(adata, mode="bar",cluster_key="cell_type")
plt.savefig('./figures/germcellforcellrank_barplot_clusterfates1225.pdf')

In [224]:
sc.settings.set_figure_params(dpi = 50, dpi_save = 600, figsize = (12, 5))
cr.pl.cluster_fates(adata, mode="heatmap",cluster_key="cell_type")
plt.savefig('./figures/germcellforcellrank_heatmap_clusterfates1225.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/scanpy/_settings.py:447: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  IPython.display.set_matplotlib_formats(*ipython_format)
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


# We use pie charts to show cell fates averaged per cluster. Edges between clusters are given by transcriptomic similarity between the clusters, just as in normal PAGA

# Compute lineage drivers
# We can compute the driver genes for all or just a subset of lineages. We can also restrict this to some subset of clusters by specifying clusters=... (not shown below). In the resulting dataframe, we also see the p-value, the corrected p-value (q-value) and the 95% confidence interval for the correlation statistic.

In [200]:
cr.tl.lineage_drivers(adata)

Adding `adata.varm['terminal_lineage_drivers']`
       `.lineage_drivers`
    Finish (0:00:00)


/tmp/ipykernel_178141/1721556810.py:1: DeprecationWarning: `cellrank.tl.lineage_drivers` will be removed in version `2.0`. Please use the `cellrank.kernels` or `cellrank.estimators` interface instead.
  cr.tl.lineage_drivers(adata)


,RS1_corr,RS1_pval,RS1_qval,RS1_ci_low,RS1_ci_high,RS2_corr,RS2_pval,RS2_qval,RS2_ci_low,RS2_ci_high,ES_corr,ES_pval,ES_qval,ES_ci_low,ES_ci_high
P2RY14,0.526016,3.106227e-289,6.212454e-286,0.502836,0.548440,-0.290365,4.139500e-77,7.199131e-76,-0.318962,-0.261240,-0.170379,1.055257e-26,5.798117e-26,-0.200817,-0.139613
HORMAD1,0.425591,1.205370e-175,1.205370e-172,0.399433,0.451057,-0.107817,1.707844e-11,4.872593e-11,-0.138861,-0.076561,-0.259316,3.785060e-61,3.963414e-60,-0.288474,-0.229677
GPR171,0.323743,8.524472e-97,5.682982e-94,0.295239,0.351671,-0.188720,1.598679e-32,1.188609e-31,-0.218933,-0.158144,-0.095296,2.807329e-09,7.198280e-09,-0.126432,-0.063972
ZNF389,0.323055,2.312485e-96,1.156242e-93,0.294538,0.350997,-0.136866,1.103817e-17,4.286666e-17,-0.167659,-0.105806,-0.144260,1.692703e-19,7.391715e-19,-0.174981,-0.113259
ATF7IP2,0.300213,1.229834e-82,4.099445e-80,0.271264,0.328620,-0.030408,5.862343e-02,8.251010e-02,-0.061864,0.001109,-0.226542,1.369229e-46,1.160364e-45,-0.256229,-0.196429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GSG1,-0.272573,1.071425e-67,1.530607e-65,-0.301499,-0.243145,-0.470936,1.032300e-221,1.474715e-219,-0.495103,-0.446040,0.682082,0.000000e+00,0.000000e+00,0.664858,0.698581
TNP1,-0.275364,3.989915e-69,6.649859e-67,-0.304240,-0.245983,-0.513164,3.329267e-272,6.658534e-269,-0.536011,-0.489566,0.724811,0.000000e+00,0.000000e+00,0.709502,0.739436
C18H7orf34,-0.276517,1.013913e-69,1.843478e-67,-0.305371,-0.247155,-0.466989,2.178849e-217,2.905132e-215,-0.491275,-0.441978,0.681669,0.000000e+00,0.000000e+00,0.664427,0.698185
PRM3,-0.292000,5.190668e-78,1.038134e-75,-0.320566,-0.262904,-0.459084,6.498908e-209,7.221009e-207,-0.483603,-0.433845,0.687297,0.000000e+00,0.000000e+00,0.670301,0.703573


# Afterwards, we can plot the top 5 driver genes (based on the correlation), e.g. for the 1 lineage

In [203]:
cr.pl.lineage_drivers(adata, lineage="ES", n_genes=10)
plt.savefig('./figures/germcellforcellrank_ESdrivergenes1225.pdf')

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


## Gene expression trends
# The functions demonstrated above are the main functions of CellRank: computing initial and terminal states and probabilistic fate maps. We can use the computed probabilities now to e.g. smooth gene expression trends along lineages.

# Let’s start with a temporal ordering for the cells. To get this, we can compute scVelo’s latent time, as computed previously, or alternatively, we can just use CellRank’s initial states to compute a [Haghverdi et al., 2016].

In [204]:
# compue DPT, starting from CellRank defined root cell
root_idx = np.where(adata.obs["initial_states"] == "SSC1")[0][0]
adata.uns["iroot"] = root_idx
sc.tl.dpt(adata)

scv.pl.scatter(
    adata,
    color=["cell_type", root_idx, "latent_time", "dpt_pseudotime"],
    fontsize=16,
    cmap="viridis",
    perc=[2, 98],
    colorbar=True,
    rescale_color=[0, 1],
    title=["cell_type", "root cell", "latent time", "dpt pseudotime"],
    save='./figures/germcellforcellrank_temporalorderings1225.pdf'
)

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


saving figure to file ./figures/germcellforcellrank_temporalorderings1225.pdf


# We can plot dynamics of genes in pseudotime along individual trajectories, defined via the fate maps we computed above.

# ['UCHL1', 'SOHLH1', 'DMRT1','STRA8','SMC1B','ZCWPW1','ACRV1','SPACA1','PRM1', 'PRM2', 'TNP1', 'TNP2', 'SPATA3']
# 'UCHL1','DMRT1','ZCWPW1','ACRV1','SPACA1','SPATA3','PRM1', 'PRM2', 'TNP1', 'TNP2'

In [215]:
model = cr.ul.models.GAM(adata)
cr.pl.gene_trends(
    adata,
    model=model,
    data_key="X",
    genes=['LOC100620512', 'OAZ3', 'GSG1'],
    ncols=3,
    time_key="latent_time",
    same_plot=True,
    hide_cells=True,
    figsize=(15, 4),
    n_test_points=200,
    save='./germcellforcellrank_LOC100620512geneexpressiontrends1225.pdf'
)


Computing trends using `1` core(s)


  0%|          | 0/3 [00:00<?, ?gene/s]

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:649: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bases = (x >= aug_knots[:-1]).astype(np.int) * \
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:650: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

    Finish (0:00:01)
Plotting trends


# We can also visualize the lineage drivers computed above in a heatmap. Below, we do this for the Alpha lineage, i.e. we smooth gene expression for the putative Alpha-drivers in pseudotime, using as cell-level weights the Alpha fate probabilities. We sort genes according to their peak in pseudotime, thus revealing a cascade of gene expression events.

In [218]:
cr.pl.heatmap(
    adata,
    model,
    genes=adata.varm['terminal_lineage_drivers']["ES_corr"].sort_values(ascending=False).index[:100],
    show_absorption_probabilities=True,
    lineages="ES",
    n_jobs=1,
    backend="loky",
    save='./germcellforcellrank_ESgeneexpressiontrendsheatmap1225.pdf'
)

Computing trends using `1` core(s)


  0%|          | 0/100 [00:00<?, ?gene/s]

/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:649: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bases = (x >= aug_knots[:-1]).astype(np.int) * \
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:650: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

did not converge


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:649: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bases = (x >= aug_knots[:-1]).astype(np.int) * \
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/pygam/utils.py:650: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your cu

    Finish (0:00:16)


/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1649: DeprecationWarning: NONE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  img = img.convert(mode='P', dither=Image.NONE,
/public/home/zhaox/anaconda3/lib/python3.9/site-packages/matplotlib/backends/backend_pdf.py:1650: DeprecationWarning: ADAPTIVE is deprecated and will be removed in Pillow 10 (2023-07-01). Use Palette.ADAPTIVE instead.
  palette=Image.ADAPTIVE, colors=num_colors)


## 保存跑完cellrank的结果文件

In [236]:
adata

AnnData object with n_obs × n_vars = 3868 × 2000
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'total_counts_chrX', 'pct_counts_chrX', 'total_counts_chrY', 'pct_counts_chrY', 'S_score', 'G2M_score', 'phase', 'doublet', 'doublet_score', 'leiden', 'cell_type', 'initial_size_spliced', 'initial_size_unspliced', 'initial_size', 'n_counts', 'velocity_self_transition', 'terminal_states', 'terminal_states_probs', 'clusters_gradients', 'initial_states', 'initial_states_probs', 'velocity_pseudotime', 'latent_time', 'dpt_pseudotime', 'to_terminal_states_kl_divergence'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'chrX', 'chrY', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'Accession', 'Chromosome', 'End', 'Start', 'Strand', 'fit_r2', 'fit_alpha', 'fit_beta', 'fit_gamma', 'fit_t_', 'fit_scaling', 'fit_std_u', 'fit_std_s', 'fit_likelihood

In [239]:
results_file = './figures/adult_germsubsetforcellrank_results1229.h5ad'
adata.write(results_file)

TypeError: No method has been defined for writing <class 'cellrank.tl._lineage.Lineage'> elements to <class 'h5py._hl.group.Group'>

Above error raised while writing key 'terminal_states_memberships' of <class 'h5py._hl.group.Group'> to /